##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemma - finetune with LLaMA Factory

This notebook demonstrates how to finetune Gemma with LLaMA Factory. [LLaMA Factory](https://github.com/InternLM/xtuner) is a tool that specifically designed for finetuning LLMs. LLaMA Factory wraps the Hugging Face finetuning functionality and provides a simple interface for finetuning. It's very easy to finetune Gemma with LLaMA Factory. This notebook follows very closely the official [Colab notebook](https://colab.research.google.com/drive/1eRTPn37ltBbYsISy9Aw2NuI2Aq5CQrD9?usp=sharing) from LLaMA Factory.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/Finetune_with_LLaMA_Factory.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.


### Gemma setup on Hugging Face
LLaMA Factory uses Hugging Face under the hood. So you will need to:

* Get access to Gemma on [huggingface.co](huggingface.co) by accepting the Gemma license on the Hugging Face page of the specific model, i.e., [Gemma 2B](https://huggingface.co/google/gemma-2b).
* Generate a [Hugging Face access token](https://huggingface.co/docs/hub/en/security-tokens) and configure it as a Colab secret 'HF_TOKEN'.

In [1]:
import os
from google.colab import userdata
# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

### Install LLaMA Factory

Install LLaMA Factory from source on GitHub.

In [2]:
%cd /content/
%rm -rf LLaMA-Factory
!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install -e .[torch,bitsandbytes]

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 20658, done.
remote: Counting objects: 100% (234/234), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 20658 (delta 176), reused 129 (delta 129), pack-reused 20424 (from 3)
Receiving objects: 100% (20658/20658), 235.49 MiB | 15.56 MiB/s, done.
Resolving deltas: 100% (14929/14929), done.
/content/LLaMA-Factory
assets/       docker/      LICENSE      pyproject.toml  requirements.txt  src/
CITATION.cff  evaluation/  Makefile     README.md       scripts/          tests/
data/         examples/    MANIFEST.in  README_zh.md    setup.py
Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for llamafactory (pyproject.toml) ... done
  Created wheel for llamafactory: filename=llamafactory-0.9.

In [7]:
import torch
torch.cuda.empty_cache()

## Finetune Gemma

Kick off Gemma 2B finetuning with a [demo Alpaca dataset](https://github.com/hiyouga/LLaMA-Factory/blob/main/data/alpaca_en_demo.json). If you want to use your own dataset, follow this [guide from LLaMA Factory](https://github.com/hiyouga/LLaMA-Factory/tree/main/data).

In [8]:
import json

args = dict(
    stage="sft",  # do supervised fine-tuning
    do_train=True,
    model_name_or_path="google/gemma-2b-it",  # use bnb-4bit-quantized Gemma 2B model
    dataset="train_transformed",  # use the demo alpaca datasets
    template="gemma",  # use Gemma prompt template
    finetuning_type="lora",  # use LoRA adapters to save memory
    lora_target="all",  # attach LoRA adapters to all linear layers
    output_dir="gemma_lora",  # the path to save LoRA adapters
    per_device_train_batch_size=1,  # the batch size
    gradient_accumulation_steps=8,  # the gradient accumulation steps
    lr_scheduler_type="cosine",  # use cosine learning rate scheduler
    logging_steps=10,  # log every 10 steps
    warmup_ratio=0.1,  # use warmup scheduler
    save_steps=1000,  # save checkpoint every 1000 steps
    learning_rate=5e-4,  # the learning rate
    num_train_epochs=3.0,  # the epochs of training
    max_samples=5000,  # use 500 examples in each dataset
    max_grad_norm=1.0,  # clip gradient norm to 1.0
    quantization_bit=4,  # use 4-bit QLoRA
    loraplus_lr_ratio=16.0,  # use LoRA+ algorithm with lambda=16.0
    fp16=True,  # use float16 mixed precision training
)

json.dump(args, open("train_gemma.json", "w", encoding="utf-8"), indent=2)

%cd /content/LLaMA-Factory/

!llamafactory-cli train train_gemma.json

/content/LLaMA-Factory
2025-01-15 12:14:35.269057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-15 12:14:35.288315: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-15 12:14:35.294223: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-15 12:14:36.386292: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[WARNING|2025-01-15 12:14:43] llamafactory.hparams.parser:162 >> We recommend enable `upcast_layernorm` in quantized training.
[INFO|2025-01-15 12:14:43] llamafactory.hparams.parser:380 >> Process rank: 0, device: cuda

## Run inference in a chat setting

In [12]:
import pandas as pd
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc

# LLaMA-Factory 경로 설정
%cd /content/LLaMA-Factory/src/

# 모델 초기화
args = dict(
    model_name_or_path="google/gemma-2b-it",  # use Gemma 2B model
    adapter_name_or_path="/content/LLaMA-Factory/gemma_lora/checkpoint-1875",  # load the saved LoRA adapters
    template="gemma",  # same to the one in training
    finetuning_type="lora",  # same to the one in training
    quantization_bit=4,  # load 4-bit quantized model
)
chat_model = ChatModel(args)

# test.csv 파일 로드
test_file_path = "/content/test.csv"  # 파일 경로
test_data = pd.read_csv(test_file_path)

# 복원 결과를 저장할 리스트
results = []


/content/LLaMA-Factory/src


[INFO|configuration_utils.py:679] 2025-01-15 14:11:38,089 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2b-it/snapshots/96988410cbdaeb8d5093d1ebdc5a8fb563e02bad/config.json
[INFO|configuration_utils.py:746] 2025-01-15 14:11:38,092 >> Model config GemmaConfig {
  "_name_or_path": "google/gemma-2b-it",
  "architectures": [
    "GemmaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "eos_token_id": 1,
  "head_dim": 256,
  "hidden_act": "gelu",
  "hidden_activation": null,
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 16384,
  "max_position_embeddings": 8192,
  "model_type": "gemma",
  "num_attention_heads": 8,
  "num_hidden_layers": 18,
  "num_key_value_heads": 1,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.46.1",
  "use_cache": true,
  "vocab_size

[INFO|2025-01-15 14:11:42] llamafactory.model.model_utils.quantization:157 >> Quantizing model to 4 bit with bitsandbytes.
[INFO|2025-01-15 14:11:42] llamafactory.model.patcher:157 >> Using KV cache for faster generation.


[INFO|modeling_utils.py:3937] 2025-01-15 14:11:42,329 >> loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--google--gemma-2b-it/snapshots/96988410cbdaeb8d5093d1ebdc5a8fb563e02bad/model.safetensors.index.json
[INFO|modeling_utils.py:1670] 2025-01-15 14:11:42,332 >> Instantiating GemmaForCausalLM model under default dtype torch.bfloat16.
[INFO|configuration_utils.py:1096] 2025-01-15 14:11:42,335 >> Generate config GenerationConfig {
  "bos_token_id": 2,
  "eos_token_id": 1,
  "pad_token_id": 0
}



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[INFO|modeling_utils.py:4800] 2025-01-15 14:11:45,221 >> All model checkpoint weights were used when initializing GemmaForCausalLM.

[INFO|modeling_utils.py:4808] 2025-01-15 14:11:45,224 >> All the weights of GemmaForCausalLM were initialized from the model checkpoint at google/gemma-2b-it.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GemmaForCausalLM for predictions without further training.
[INFO|configuration_utils.py:1051] 2025-01-15 14:11:45,678 >> loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2b-it/snapshots/96988410cbdaeb8d5093d1ebdc5a8fb563e02bad/generation_config.json
[INFO|configuration_utils.py:1096] 2025-01-15 14:11:45,680 >> Generate config GenerationConfig {
  "bos_token_id": 2,
  "eos_token_id": 1,
  "pad_token_id": 0
}



[INFO|2025-01-15 14:11:45] llamafactory.model.model_utils.attention:157 >> Using torch SDPA for faster training and inference.
[INFO|2025-01-15 14:11:46] llamafactory.model.adapter:157 >> Loaded adapter(s): /content/LLaMA-Factory/gemma_lora/checkpoint-1875
[INFO|2025-01-15 14:11:46] llamafactory.model.loader:157 >> all params: 2,515,978,240


In [17]:
from tqdm import tqdm

instruction = "Restore the following phonetically encoded text to its original meaning."
print("Processing inputs...")

# tqdm와 함께 itertuples() 사용으로 성능 최적화
for row in tqdm(test_data.itertuples(index=False), total=len(test_data), mininterval=0.01):
    input_text = row.input  # itertuples()는 속성 접근 방식을 사용
    query = f"{instruction}\nInput: {input_text}"

    messages = [{"role": "user", "content": query}]
    response = ""

    # 모델 추론
    for new_text in chat_model.stream_chat(messages):
        response += new_text
        print(response)

    # 결과 저장
    results.append({"ID": row.ID, "output": response.strip()})  # itertuples에서는 ID도 속성 접근

    # 메모리 관리
    torch_gc()

print("All inputs processed.")


Processing inputs...


  0%|          | 0/1689 [23:31<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# 결과를 DataFrame으로 변환
submission_df = pd.DataFrame(results)

# sample_submission.csv 형식에 맞게 저장
output_file_path = "/content/sample_submission.csv"
submission_df.to_csv(output_file_path, index=False)

print(f"Results saved to {output_file_path}")


## Merge the LoRA adapter and upload the finetuned model to Hugging Face

In [9]:
import json

args = dict(
    model_name_or_path="google/gemma-2b-it",  # use official non-quantized Gemma 2B model
    adapter_name_or_path="gemma_lora",  # load the saved LoRA adapters
    template="gemma",  # same to the one in training
    finetuning_type="lora",  # same to the one in training
    export_dir="gemma_lora_merged",  # path to save the merged model
    export_size=2,  # the file shard size (in GB) of the merged model
    export_device="cpu",  # the device used in export, can be chosen from `cpu` and `cuda`
    export_hub_model_id="gemma-2b-if-finetuned-decode-kor",  # your Hugging Face hub model ID
)

json.dump(args, open("merge_gemma.json", "w", encoding="utf-8"), indent=2)

%cd /content/LLaMA-Factory/

!llamafactory-cli export merge_gemma.json

/content/LLaMA-Factory
2025-01-15 13:59:11.526757: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-15 13:59:11.560856: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-15 13:59:11.571109: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-15 13:59:13.152608: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[INFO|configuration_utils.py:679] 2025-01-15 13:59:19,010 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2b-it/snapshots/96988410cbdaeb8d5093d1ebdc5a8fb563e02